## 2.5. 인코더

```mermaid
flowchart TD
    input[입력]
    token_embedding[토큰 임베딩]
    position_embedding[위치 임베딩]
    layer_norm[층 정규화]
    multi_head_attention[멀티 헤드 어텐션]
    layer_norm2[층 정규화]
    feed_forward[피드 포워드 신경망]
    layer_norm3[층 정규화]

    input --> token_embedding
    token_embedding --> position_embedding
    position_embedding --> layer_norm
    subgraph "인코더 층 반복"
        layer_norm --> multi_head_attention
    multi_head_attention --> A
    A@{ shape: cross-circ, label: "Summary" } --> layer_norm2
        layer_norm2 --> feed_forward
    end
    A --> layer_norm
    feed_forward --> B
    B@{ shape: cross-circ, label: "Summary" } --> layer_norm3
    B --> A
    layer_norm3
```

In [6]:
# 예제 2.11 인코더층

import torch.nn as nn

class PreLayerNormFeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward, dropout):
        super().__init__()
        self.linear1 = nn.Linear(d_model, dim_feedforward) # 1번째 선형 층
        self.linear2 = nn.Linear(dim_feedforward, d_model) # 2번째 선형 층
        self.dropout1 = nn.Dropout(dropout) # 드롭아웃 1
        self.dropout2 = nn.Dropout(dropout) # 드롭아웃 2
        self.activation = nn.GELU() # 활성화 함수
        self.norm = nn.LayerNorm(d_model) # 층 정규화

    def forward(self, src):
        x = self.norm(src)
        x = x + self.linear2(self.activation(self.linear1(x))) # 2개의 선형 층과 활성화 함수
        x = self.dropout2(x) # 드롭아웃 2
        return x


class TransformerEncoderLayer(nn.Module):
    """
    TransformerEncoderLayer 클래스는 트랜스포머 인코더의 한 층을 구현합니다.
    초기화 메서드:
    메서드:
        forward(src):
            주어진 입력에 대해 트랜스포머 인코더 층을 연산합니다.
                src (Tensor): 입력 텐서.
            Returns:
                Tensor: 트랜스포머 인코더 층을 통과한 출력 텐서.
    """
    def __init__(self, d_model, n_head, dim_feedforward, dropout):
        """
        초기화 메서드
        Args:
            d_model (int): 모델의 차원.
            n_head (int): 멀티헤드 어텐션의 헤드 수.
            dim_feedforward (int): 피드포워드 신경망의 차원.
            dropout (float): 드롭아웃 확률.
        """
        
        super().__init__()
        self.atten = nn.MultiheadAttention(d_model, d_model, n_head) # 멀티헤드 어텐션 층
        self.norm1 = nn.LayerNorm(d_model) # 층 정규화
        self.dropout1 = nn.Dropout(dropout) # 드롭아웃
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드포워드 신경망

    def forward(self, src):
        norm_x = self.norm1(src) # norm1을 적용해 층 정규화
        atten_output = self.atten(norm_x, norm_x, norm_x) # 멀티헤드 어텐션을 연산
        x = src + self.dropout1(atten_output) # 드롭아웃을 적용해 어텐션 출력을 더함
        x = self.feed_forward(x) # 피드포워드 신경망을 연산
        return x

In [4]:
# 예제 2.12 인코더 구현

import copy

embedding_dim = 16 # 임베딩 차원
norm = nn.LayerNorm(embedding_dim)

def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm

    def forward(self, src):
        output = src
        for mod in self.layers:
            output = mod(output)
        return self.norm(output)